# GR1A example

In [10]:
from pathlib import Path
import pandas as pd
import datetime
from hydrogr import InputDataHandler, ModelGr1a
import plotly.graph_objects as go

## Prepare the input data

Here we load a dataframe that contains daily rainfall and evapotranspiration data.
To be sued with the GR1A model, we need this data to be resampled to an annual period.

In [11]:
# Prepare inputs data :
data_path = Path.cwd().parent / "data"
df = pd.read_pickle(data_path / "L0123001.pkl")
df.columns = ["date", "precipitation", "temperature", "evapotranspiration", "flow", "flow_mm"]
df.index = df["date"]

# One year aggregation using period as index :
precipitation = df["precipitation"].resample("A", kind="period").sum()
temperature = df["temperature"].resample("A", kind="period").mean()
evapotranspiration = df["evapotranspiration"].resample("A", kind="period").sum()
flow_mm = df["flow_mm"].resample("A", kind="period").sum()
df = pd.concat([precipitation, temperature, evapotranspiration, flow_mm], axis=1)
df["date"] = df.index

For demonstration purpose, we will not run the model on all the dataset period, but on a sub-period from 1989 to 1999.

In [12]:
inputs = InputDataHandler(ModelGr1a, df)
start_date = datetime.datetime(1989, 1, 1, 0, 0)
end_date = datetime.datetime(1999, 1, 1, 0, 0)
inputs = inputs.get_sub_period(start_date, end_date)

## Setup the model

The GR1A model only take one parameter : the store capacity in mm

In [13]:
parameters = [0.840]
model = ModelGr1a(parameters)

## Run the model and display the results

In [14]:
outputs = model.run(inputs.data)
print(outputs.head())

                  flow
date                  
1989-01-01    0.000000
1990-01-01  521.427432
1991-01-01  701.322905
1992-01-01  672.571346
1993-01-01  729.165314


In [15]:
fig = go.Figure([
    go.Scatter(x=outputs.index, y=outputs['flow'], name="Calculated"),
    go.Scatter(x=inputs.data.index, y=inputs.data['flow_mm'], name="Observed"),
])
fig.show()